In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/content/dataoriginal/lifebeamdata.csv')

df = df.round(2)

df.to_csv('lifebeamdataNew.csv', index=False)

df['BMI'] = df['Weight'] / (df['Height'] ** 2)

df.loc[df['BMI'] >= 25, 'CNO'] = 0
df.loc[df['BMI'] < 25, 'CNO'] = (25 * (df.loc[df['BMI'] < 25, 'Height'] ** 2) - df.loc[df['BMI'] < 25, 'Weight']) * 7700

df.loc[df['BMI'] <= 18.49, 'CLU'] = 0
df.loc[df['BMI'] > 18.49, 'CLU'] = (df.loc[df['BMI'] > 18.49, 'Weight'] - (18.49 * (df.loc[df['BMI'] > 18.49, 'Height'] ** 2))) * (-7700)

df['BMR'] = df.apply(lambda row: 66 + (13.7 * row['Weight']) + (5 * row['Height']) - (6.8 * row['Age'])
if row['Gender'] == '0'
else 655 + (9.6 * row['Weight']) + (1.8 * row['Height']) - (4.7 * row['Age']), axis=1)

df['Cintake'] = df['Calories']/df['Day']
df['Cburned'] = df['Weight']*df['Exercise']/df['Day']

df['Eused'] = df['BMR'] * 1.3

df['sisaCalories'] = df['Cintake'] - df['Cburned'] - df['Eused']


In [ ]:
df.to_csv('cleanedDataBeta.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('cleanedDataBeta.csv')

df = df.round(2)

df.to_csv('roundedCleanedDataBeta.csv', index=False)

MODELING

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error
import pickle

df = pd.read_csv('roundedCleanedDataBeta.csv')

X = df.drop(columns=['sisaCalories'])
y = df['sisaCalories']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

scaler_filename = 'scalerBeta.pkl'
pickle.dump(scaler, open(scaler_filename, 'wb'))

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

model.fit(X_train_scaled, y_train, epochs=1000, batch_size=32, validation_split=0.2)

model.save('ModelBeta.h5')

y_pred = model.predict(X_test_scaled).flatten()

print('R2 score', r2_score(y_test, y_pred))
print('MAE', mean_absolute_error(y_test, y_pred))


Epoch 1/1000
3/3 [==============================] - 2s 169ms/step - loss: 1232761.5000 - val_loss: 765493.5000
Epoch 2/1000
3/3 [==============================] - 0s 25ms/step - loss: 1232389.2500 - val_loss: 765253.8750
Epoch 3/1000
3/3 [==============================] - 0s 23ms/step - loss: 1232055.5000 - val_loss: 765019.1875
Epoch 4/1000
3/3 [==============================] - 0s 24ms/step - loss: 1231735.0000 - val_loss: 764784.5000
Epoch 5/1000
3/3 [==============================] - 0s 23ms/step - loss: 1231392.6250 - val_loss: 764554.4375
Epoch 6/1000
3/3 [==============================] - 0s 38ms/step - loss: 1231052.0000 - val_loss: 764319.5000
Epoch 7/1000
3/3 [==============================] - 0s 28ms/step - loss: 1230705.0000 - val_loss: 764070.7500
Epoch 8/1000
3/3 [==============================] - 0s 45ms/step - loss: 1230351.5000 - val_loss: 763812.4375
Epoch 9/1000
3/3 [==============================] - 0s 48ms/step - loss: 1229944.7500 - val_loss: 763543.6250
Epoch 10/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = tf.keras.models.load_model('/content/ModelBeta.h5')

scaler_filename = '/content/scalerBeta.pkl'
scaler = pickle.load(open(scaler_filename, 'rb'))

new_data = {
    'Gender': 0,
    'Age': 35,
    'Height': 1.8,
    'Weight': 90,
    'Exercise': 0.721,
    'Calories': 3992,
    'Day': 2,
    'BMI': 27.7777777777777,
    'CNO': 0,
    'CLU': -231711.479999999,
    'BMR': 1357.74,
    'Cintake': 1996,
    'Cburned': 32.445,
    'Eused': 1765.062
}

new_df = pd.DataFrame([new_data])

new_data_scaled = scaler.transform(new_df)

prediction = model.predict(new_data_scaled).flatten()[0]

print('Predicted sisaCalories:', prediction)


1/1 [==============================] - 0s 54ms/step
Predicted sisaCalories: 186.82892
